In [46]:
import sqlite3
import pandas as pd
import os

def count_db_entries(db_path, table_name):
    """
    Connects to an SQLite database and counts the total number of entries,
    and provides a breakdown of entries with valid and invalid timestamps.

    Args:
        db_path (str): The path to the SQLite database file.
        table_name (str): The name of the table to inspect.
    """
    if not os.path.exists(db_path):
        print(f"Error: Database file not found at '{db_path}'")
        return

    print(f"--- Analyzing '{db_path}' ---")

    try:
        with sqlite3.connect(db_path) as conn:
            # Use a direct SQL query to get the total count efficiently
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            total_entries = cursor.fetchone()[0]
            
            if total_entries == 0:
                print("The table is empty.")
                return

            # For NaT analysis, it's easier to use pandas
            df = pd.read_sql_query(f"SELECT timestamp FROM {table_name}", conn)

            # Convert timestamp to numeric, coercing errors to NaT (Not a Time)
            numeric_timestamps = pd.to_numeric(df['timestamp'], errors='coerce')
            
            # Count the NaT entries by checking where the conversion resulted in NaN
            nat_entries = numeric_timestamps.isna().sum()
            valid_entries = total_entries - nat_entries
            
            print("\n--- Entry Report ---")
            print(f"Total entries in the table: {total_entries}")
            print("------------------------")
            print(f"Entries with valid timestamps:   {valid_entries}")
            print(f"Entries with invalid timestamps (NaT): {nat_entries}")
            print("------------------------")

    except Exception as e:
        print(f"An error occurred: {e}")

def display_newest_entries(db_path, table_name, num_entries=10):
    """
    Connects to an SQLite database, reads the data, and displays the
    most recent entries based on the timestamp.

    Args:
        db_path (str): The path to the SQLite database file.
        table_name (str): The name of the table to inspect.
        num_entries (int): The number of newest entries to display.
    """
    if not os.path.exists(db_path):
        # This check is redundant if called after count_db_entries, but good practice
        return 

    print(f"\n--- Displaying {num_entries} Newest Entries from '{table_name}' ---")

    try:
        with sqlite3.connect(db_path) as conn:
            # Read the entire table into a pandas DataFrame
            # Using SELECT * to get all columns for display
            df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

            if df.empty:
                print("The table is empty. Cannot display entries.")
                return
            
            # Ensure the timestamp column is numeric to allow for proper sorting
            # Invalid timestamps will become 'NaN' (Not a Number)
            df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')

            # Drop rows with invalid timestamps before sorting
            df.dropna(subset=['timestamp'], inplace=True)

            if df.empty:
                print("No entries with valid timestamps found.")
                return
                
            # Sort the DataFrame by timestamp in descending order (newest first)
            df_sorted = df.sort_values(by='timestamp', ascending=False)
            
            # Get the top 'num_entries' rows
            newest_entries = df_sorted.head(num_entries)
            
            print(newest_entries)
            print("----------------------------------------------------")

    except Exception as e:
        print(f"An error occurred while displaying entries: {e}")


# --- Main execution block ---
if __name__ == "__main__":
    DATABASE_FILE_PATH = 'selene-db'
    TABLE_NAME = 'SampleEeg'
    
    # First, run your original analysis
    count_db_entries(DATABASE_FILE_PATH, TABLE_NAME)
    
    # Second, display the 15 newest entries from the latest session
    display_newest_entries(DATABASE_FILE_PATH, TABLE_NAME, num_entries=15)

--- Analyzing 'selene-db' ---

--- Entry Report ---
Total entries in the table: 104224
------------------------
Entries with valid timestamps:   104224
Entries with invalid timestamps (NaT): 0
------------------------

--- Displaying 15 Newest Entries from 'SampleEeg' ---
            timestamp  channel_c1  channel_c2  channel_c3  channel_c4  \
104223  1751060658981  211579.650  214615.485  220280.580  216740.520   
104222  1751060658980  222741.270  225767.610  231368.715  227785.815   
104221  1751060658977  224926.290  227883.195  233375.445  229693.095   
104220  1751060658973  216484.425  219408.030  224894.835  221188.770   
104219  1751060658972  211971.780  214887.600  220408.065  216718.605   
104218  1751060658971  208372.725  211303.395  216881.280  213236.010   
104217  1751060658970  206379.990  209341.980  214982.910  211393.755   
104216  1751060658965  205493.445  208492.875  214182.720  210642.345   
104215  1751060658961  211574.520  214610.400  220275.360  216736.020 